In [32]:
import yaml
import pandas as pd
import os
import logging
from imblearn.over_sampling import SMOTE
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pickle

In [13]:
%pwd

'/Users/pratik.kujur/Desktop/Projects/Mlops-Classification-End-to-end/experiments'

In [14]:
os.chdir("../")
%pwd

'/Users/pratik.kujur/Desktop/Projects/Mlops-Classification-End-to-end'

In [9]:
params_preprocess=yaml.safe_load(open("/Users/pratik.kujur/Desktop/Projects/Mlops-Classification-End-to-end/params.yaml"))['preprocess']

In [19]:
logging.basicConfig(level=logging.INFO, format='[%(asctime)s]: %(message)s:')

In [28]:
class DataPreprocessing:
    def __init__ (self,InputPath,OutputPath,XsmPath,ysmPath):
        self.input_path=InputPath
        self.output_path=OutputPath
        self.Xsm_path=XsmPath
        self.ysm_path=ysmPath

    def savefiles(self,file,path):
        os.makedirs(os.path.dirname(path),exist_ok=True)
        file.to_csv(path,index=False,header=True)

    def DataPreprocess(self):
        try:
            df=pd.read_csv(self.input_path)
            df.drop(columns=['id','Unnamed: 32'],inplace=True,axis="columns")
            df['diagnosis']=df['diagnosis'].map({'M':0,'B':1})
            self.savefiles(df,self.output_path)
            
            #feature and target split
            y=df.iloc[:,0:1]
            X=df.iloc[:,1:]

            #handling data imbalance
            class_balancer=SMOTE()
            X_sm,y_sm=class_balancer.fit_resample(X,y)
            self.savefiles(X_sm,self.Xsm_path)
            self.savefiles(y_sm,self.ysm_path)


            logging.info("Data preprocessed successfully.")

            
        except Exception as e:
            logging.exception("Exception occurred: %s", str(e))
            raise e

In [29]:
DataIngestion=DataPreprocessing(params_preprocess['input'],params_preprocess['output'],params_preprocess['X_sm'],params_preprocess['y_sm'])
DataIngestion.DataPreprocess()

In [40]:
params_trainer=yaml.safe_load(open("/Users/pratik.kujur/Desktop/Projects/Mlops-Classification-End-to-end/params.yaml"))['train']
params_eval=yaml.safe_load(open("/Users/pratik.kujur/Desktop/Projects/Mlops-Classification-End-to-end/params.yaml"))['evaluation']
params_model=yaml.safe_load(open("/Users/pratik.kujur/Desktop/Projects/Mlops-Classification-End-to-end/params.yaml"))

In [56]:
class ModelTrainer:
    def __init__ (self,X,Y,X_train,y_train,X_test,y_test,model_path):
        self.X=X
        self.y=Y
        self.X_train_path=X_train
        self.y_train_path=y_train
        self.y_test_path=y_test
        self.X_test_path=X_test
        self.ModelPath=model_path
        

    def savefiles(self,file,path):
        os.makedirs(os.path.dirname(path),exist_ok=True)
        file.to_csv(path,index=False,header=True)

    def model_trainer(self,X,y,ModelPath):
        clf=RandomForestClassifier(criterion='entropy',max_features='sqrt')
        clf.fit(X,y)
        os.makedirs(os.path.dirname(ModelPath),exist_ok=True)
        pickle.dump(clf,open(ModelPath,'wb'))


    def StartTraining(self):
        try:
            X=pd.read_csv(self.X)
            y=pd.read_csv(self.y)

            X_train, X_test, y_train, y_test=train_test_split(X,y,random_state=42,test_size=0.2)

            saving_files=[
                           (X_train,self.X_train_path),
                           (X_test,self.y_train_path), 
                           (y_train,self.y_test_path), 
                           (y_test,self.X_test_path)
                        ]
            for file,path in (saving_files):
                self.savefiles(file,path)
            logging.info("Data Splited and Saved successfully.")

            self.model_trainer(X_train,y_train,self.ModelPath)
            logging.info("Model Trained and Saved successfully.")




            
        except Exception as e:
            logging.exception("Exception occurred: %s", str(e))
            raise e

In [57]:
ModelTraining=ModelTrainer(params_preprocess['X_sm'],params_preprocess['y_sm'],
                           params_trainer['X_train'],params_trainer['y_train'],
                           params_eval['X_test'],params_eval['y_test'],
                           params_model['model'])
ModelTraining.StartTraining()

/Users/pratik.kujur/Desktop/Projects/Mlops-Classification-End-to-end/.venv/lib/python3.10/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
